In [ ]:
# conda create -n pytorch3d python=3.9
# conda activate pytorch3d
# conda install pytorch=1.13.0 torchvision pytorch-cuda=11.6 -c pytorch -c nvidia
# conda install -c fvcore -c iopath -c conda-forge fvcore iopath

In [26]:
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from math import radians
from pytorch3d.transforms import axis_angle_to_matrix
# Util function for loading point clouds|
import numpy as np

# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_rotation,
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor,
    FoVPerspectiveCameras,
    PerspectiveCameras,
    rotate_on_spot,
    OrthographicCameras
)

ModuleNotFoundError: No module named 'pytorch3d'

In [22]:
import open3d as o3d
import torch
import torch.nn as nn
import numpy as np
import torchvision
from Normalization import MeshNormalizer,PointCloudNormalizer

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

class point_cloudx():
  def __init__(self,verts,rbg_colors):
    self.vertices=verts;
    self.color=rbg_colors;
  def printxx(self):
     print(self.vertices.shape[0])

pointcloud = o3d.io.read_point_cloud('../point_cloud_dataset/giraffe.ply')

# Get vertex positions
verts = torch.Tensor(np.asarray(pointcloud.points)).to(device)
manual_color = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device=device)
rgb_colors = manual_color.repeat(verts.shape[0], 1)

#point_cloud = Pointclouds(points=[verts], features=[rgb_colors])
pointcloud_classe=point_cloudx(verts,rgb_colors)
#print(pointcloud_classe.printxx())
print(pointcloud_classe.vertices[0])
PointCloudNormalizer(pointcloud_classe)()
print(pointcloud_classe.vertices[0])

point_cloud = Pointclouds(points=[verts], features=[rgb_colors])

R, T = look_at_view_transform(3, 0, 0)

angles = [radians(0), radians(0),radians(0)]
rotation = axis_angle_to_matrix(torch.FloatTensor(angles))
R, T = rotate_on_spot(R, T, rotation)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T, znear=0.01)#questa la fa di fronte

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
raster_settings = PointsRasterizationSettings(
    image_size=512,
    radius = 0.003,
    points_per_pixel = 10
)
renderer = PointsRenderer(
    rasterizer=PointsRasterizer(cameras=cameras, raster_settings=raster_settings),
    compositor=NormWeightedCompositor()
)


images = renderer(point_cloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off");

tensor([-0.0844, -0.1738, -0.2212])
tensor([-0.0843, -0.1761, -0.2207])


NameError: name 'Pointclouds' is not defined